In [54]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-semi-pred/covid-semi_pred.csv
/kaggle/input/d/ishangarg12/irdata/dataset.xlsx


In [55]:
!pip install simpletransformers transformers sacremoses

In [61]:
# df = pd.read_excel("/kaggle/input/d/ishangarg12/irdata/dataset.xlsx")
df = pd.read_excel("/kaggle/input/d/ishangarg12/irdata/dataset.xlsx")
df.head()

,Unnamed: 0.1,Unnamed: 0,Article title,Article keywords,Article abstract,Contextual
0,0,0,Assessing the impacts of COVID-19 vaccination ...,Affordability;COVID-19 | SARS-CoV-2;Decision-m...,The COVID-19 vaccine supply shortage in 2021 c...,1.0
1,1,1,Association between interleukin-10 gene polymo...,COVID-19;Interleukin-10 gene polymorphisms;SAR...,Polymorphisms in the interleukin-10 (IL10) gen...,1.0
2,2,2,Quality of Life of early-stage breast-cancer p...,Breast;COVID-19;Cancer;EORTC;Oncology;Quality ...,Objectives To describe the Quality of Life (QO...,1.0
3,3,3,"The research interest, capacity and culture of...",Barriers;Health Research;Innovation;Motivators...,The UK National Health Service (NHS) is ideall...,0.0
4,4,4,Machine learning prediction for COVID-19 disea...,COVID-19;Classification;Laboratory markers;Mac...,Early prognostication of patients hospitalized...,1.0


In [62]:

def concatenate(title, keyword, abstract):
    title = f"{title}"
    if type(keyword) == float:
        # print("Keyword: ", keyword)
        keyword = ""
    else:
        keyword = keyword.replace(";", " ")
        keyword = f"{keyword}."
    abstract = f"{abstract}"

    return title + " " + keyword + " " + abstract

In [63]:
df.dropna(subset=["Article title", "Article abstract", "Contextual"], inplace=True)
df.fillna({"Contextual": -1}, inplace=True)


df["Concatenated"] = df.apply(
    lambda x: concatenate(
        x["Article title"], x["Article keywords"], x["Article abstract"]
    ),
    axis=1,
)
df = df.astype({"Contextual": "int32"})
df = df[["Concatenated", "Contextual"]]

In [44]:
# true_pred_split = 188
# df = df[0:true_pred_split]
# # test_df = df[true_pred_split:]
# len(df)

188

In [64]:
from sklearn.model_selection import train_test_split
splits = [0.2, 0.3, 0.4, 0.5]
train_dfs = [] 
eval_dfs = []
for split in splits:
    train_df, eval_df = train_test_split(df, test_size=split, random_state=84)
    # train_df = pd.concat([train_df, test_df])
    # train_df.reset_index(drop=True, inplace=True)
    train_df = pd.DataFrame(
            {
                "text": train_df["Concatenated"].replace(r"\n", " ", regex=True),
                "label": train_df["Contextual"],
            },
            columns=["text", "label"],
        )

    train_df.reset_index(drop=True, inplace=True)

    # print(train_df.head())

    eval_df = pd.DataFrame(
            {
                "text": eval_df["Concatenated"].replace(r"\n", " ", regex=True),
                "label": eval_df["Contextual"],
            },
            columns=["text", "label"],
        )

    eval_df.reset_index(drop=True, inplace=True)
    train_dfs.append(train_df)
    eval_dfs.append(eval_df)

In [66]:
for eval_df in train_dfs:
    print(len(eval_df))

150
131
112
94


In [47]:
from simpletransformers.classification import ClassificationModel

In [19]:
# import shutil
# shutil.rmtree("/kaggle/working/outputs")
# shutil.rmtree("/kaggle/working/runs")
# shutil.rmtree("/kaggle/working/cache_dir")

In [48]:
args = {
    'no_cache': True,
    'no_save': True,
    'num_train_epochs': 8,
    'use_early_stopping': True,
    'evaluate_during_training': True,
    'overwrite_output_dir': True
}

In [50]:
def train(model, train_df, eval_df):
    # Train the model

    model.train_model(train_df, eval_df = eval_df)
def evaluate(model, eval_df):
    # Evaluate the model
    result, model_outputs, wrong_predictions = model.eval_model(eval_df)

    print(result)

    accuracy = (result["tp"] + result["tn"]) / (result["tp"] + result["tn"] + result["fp"] + result["fn"])

    precision = result['tp']/(result['tp'] + result['fp'])

    recall = result['tp']/(result['tp'] + result['fn'])

    f1 = (precision*recall)/(precision+recall)
    
    output = f"\nAccuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1}\n"

    return output

In [51]:
outputs = []

In [52]:
for i in range(0, len(splits)):
    model = ClassificationModel('roberta','roberta-base', num_labels=2, use_cuda=True, args=args)
    train_df = train_dfs[i]
    eval_df = eval_dfs[i]
    print(type(train_df))
    model.train_model(train_df, eval_df=eval_df)
    output = evaluate(model, eval_df)
    output = f"------------------RESULTS FOR {int(100-100*splits[i])}-{int(100*splits[i])} TRAIN-EVAL SPLIT------------------------\n" + output
    outputs.append(output)
    

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

<class 'pandas.core.frame.DataFrame'>


/opt/conda/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/94 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/94 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/12 [00:00<?, ?it/s]

{'mcc': 0.7358044393635583, 'tp': 50, 'tn': 32, 'fp': 7, 'fn': 5, 'auroc': 0.9375291375291375, 'auprc': 0.9493309865209736, 'eval_loss': 0.8877454888521848}


In [53]:
for output in outputs:
    print(output)
    print('--------------------------')

------------------RESULTS FOR 80-20 TRAIN-EVAL SPLIT------------------------

Accuracy: 0.9473684210526315
Precision: 1.0
Recall: 0.9090909090909091
F1 Score: 0.47619047619047616

--------------------------
------------------RESULTS FOR 70-30 TRAIN-EVAL SPLIT------------------------

Accuracy: 0.8771929824561403
Precision: 0.875
Recall: 0.9459459459459459
F1 Score: 0.45454545454545453

--------------------------
------------------RESULTS FOR 60-40 TRAIN-EVAL SPLIT------------------------

Accuracy: 0.881578947368421
Precision: 0.8723404255319149
Recall: 0.9318181818181818
F1 Score: 0.4505494505494506

--------------------------
------------------RESULTS FOR 50-50 TRAIN-EVAL SPLIT------------------------

Accuracy: 0.8723404255319149
Precision: 0.8771929824561403
Recall: 0.9090909090909091
F1 Score: 0.4464285714285714

--------------------------
